In [1]:
!pip install beir --quiet
!pip install datasets --quiet
!pip install IProgress --quiet

     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 8.4 MB 15.7 MB/s 
     |████████████████████████████████| 379 kB 44.2 MB/s 
     |████████████████████████████████| 4.9 MB 39.6 MB/s 
     |████████████████████████████████| 3.1 MB 36.4 MB/s 
     |████████████████████████████████| 3.3 MB 53.3 MB/s 
     |████████████████████████████████| 1.2 MB 37.2 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 50.8 MB/s 
     |████████████████████████████████| 895 kB 28.2 MB/s 
     |████████████████████████████████| 290 kB 5.4 MB/s 
     |████████████████████████████████| 132 kB 51.4 MB/s 
     |████████████████████████████████| 243 kB 38.0 MB/s 
     |████████████████████████████████| 1.1 MB 38.4 MB/s 
     |████████████████████████████████| 160 kB 47.1 MB/s 
     |████████████████████████████████| 192 kB 53.7 MB/s 
     |███████████████

# NLP partie 3

Création et sauvegarde du dataset

In [15]:
import pandas as pd
from datasets import load_dataset
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from sentence_transformers import SentenceTransformer
import sentence_transformers
import numpy as np

import pickle
import random
import os

dataset_name = "dataset"

In [3]:
def load_dataset_squad_v2():
    return pd.DataFrame(load_dataset("squad_v2", split='validation'))

def load_dbpedia():
    dataset = "dbpedia-entity"
    url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
    data_path = util.download_and_unzip(url, "datasets")
    return GenericDataLoader(data_folder=data_path).load(split="test")

def add_context_to_squad(corpus, squad_v2, n=10000) -> pd.DataFrame:
    corpus_keys = list(corpus.keys())
    contexts = squad_v2
    for i in range(n):
        choice = random.choice(corpus_keys)
        while len(corpus[choice]['text'].split()) < 50:
            choice = random.choice(corpus_keys)
        contexts = contexts.append({"id" : None, "title" : corpus[choice]['title'], "context" : corpus[choice]['text'], "question" : "", "answers" : {'text': [], 'answer_start': []}}, ignore_index=True)
    return contexts

def create_dataset() -> pd.DataFrame:
    if os.path.exists(dataset_name):
       with open(dataset_name, 'br') as f:
            return pickle.load(f)
    else:
        squad_v2 = load_dataset_squad_v2()
        corpus, queries, qrels = load_dbpedia()
        dataset = add_context_to_squad(corpus, squad_v2)
        with open(dataset_name, 'bw') as f:
            pickle.dump(dataset, f)
        return dataset

dataset = create_dataset()

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


datasets/dbpedia-entity.zip:   0%|          | 0.00/610M [00:00<?, ?iB/s]

  0%|          | 0/4635922 [00:00<?, ?it/s]

In [4]:
dataset

,id,title,context,question,answers
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'Franc..."
1,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 1..."
2,56ddde6b9a695914005b962a,Normans,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"{'text': ['Denmark, Iceland and Norway', 'Denm..."
3,56ddde6b9a695914005b962b,Normans,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],..."
4,56ddde6b9a695914005b962c,Normans,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,"{'text': ['10th century', 'the first half of t..."
...,...,...,...,...,...
21868,None,Agatston score,"In diagnostic cardiology, the Agatston score, ...",,"{'text': [], 'answer_start': []}"
21869,None,Adesh Samaroo,Adesh Samaroo is a Trinidadian chutney soca si...,,"{'text': [], 'answer_start': []}"
21870,None,Dedwyddfa,"Dedwyddfa is a house in Bryn Goodman, Ruthin, ...",,"{'text': [], 'answer_start': []}"
21871,None,2nd Regiment of Grochow Uhlans,The Second Regiment of Grochow Uhlans of Gener...,,"{'text': [], 'answer_start': []}"


Création des embeddings des contextes

In [5]:
model = SentenceTransformer('msmarco-roberta-base-v3')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/678 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
contexts_embeddings_fname = "contexts_embeddings"

if os.path.exists(contexts_embeddings_fname):
  with open(contexts_embeddings_fname, 'br') as f:
    contexts_embeddings = pickle.load(f)
else:
  with open(contexts_embeddings_fname, 'bw') as f:
    contexts_embeddings = model.encode(dataset['context'], convert_to_numpy=True)
    pickle.dump(contexts_embeddings, f)
    

On trouve les topK pour notre réponse

In [38]:
question = "Where do I live?"

In [39]:
def TopK(x, k):
    a = dict([(i, j) for i, j in enumerate(x)])
    sorted_a = dict(sorted(a.items(), key = lambda kv:kv[1], reverse=True))
    indices = list(sorted_a.keys())[:k]
    values = list(sorted_a.values())[:k]
    return (values, indices)

In [40]:

def answer_topk(question, top_k=5):
  top_k = min(top_k, len(dataset['context']))

  query_embedding = model.encode(question, convert_to_numpy=True)
  # We use cosine-similarity and torch.topk to find the highest 5 scores
  cos_scores = sentence_transformers.util.cos_sim(query_embedding, contexts_embeddings)[0]
  top_results = TopK(cos_scores, top_k)  # np.argpartition(cos_scores, -top_k)[-top_k:]

  print(f"Question: {question}")
  print(f"Top 5 answer:\n")

  for score, idx in zip(top_results[0], top_results[1]):
      print(dataset['context'].iloc[idx])

answer_topk(question)

Question: Where do I live?
Top 5 answer:

"Now I Lay Me" is a short story by American author Ernest Hemingway.The short story, "Now I Lay Me" is a part of Ernest Hemingway's collection of short stories entitled Men Without Women, which was published in 1927. "Now I Lay Me" takes place in war torn Europe, specifically in Italy. Many of Ernest Hemingway's stories take place in Italy due to his extensive experiences there. "Now I Lay Me" is a short story about two men sleeping in a tent.
The French population numbered about 75,000 and was heavily concentrated along the St. Lawrence River valley, with some also in Acadia (present-day New Brunswick and parts of Nova Scotia, including Île Royale (present-day Cape Breton Island)). Fewer lived in New Orleans, Biloxi, Mississippi, Mobile, Alabama and small settlements in the Illinois Country, hugging the east side of the Mississippi River and its tributaries. French fur traders and trappers traveled throughout the St. Lawrence and Mississippi w

In [34]:
dataset['context'].tail().iloc[0]

'In diagnostic cardiology, the Agatston score, named after its developer Arthur Agatston, is a measure of calcium generally included in the results from a CT Test for Coronary Calcification.The Agatston score is derived from the work of Drs. Agatston and Janowitz of the University of Miami School of Medicine and dates back into the 1980s. The original work was based on electron beam computed tomography (also known as ultrafast CT or EBCT).'